In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
import pandas as pd
from pathlib import Path
import src.mapping as mapp

nltk.download('framenet_v17')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package framenet_v17 to /root/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Esercitazione 2

In questa esercitazione vedremo come effettuare un task di allineamento tra 2 risorse lessicali: *FrameNet* e *WordNet*. Nello specifico vederemo come mappare gli "slot" di un frame ad un WordNet synset.

L'esercitazione si divide in due 3 principali:

* Annotazione manuale per creare una risorsa "Gold Standard".
* Definizione di una procedura automatica di mapping.
* Valutazione tra il mapping automatico e quello manuale.

### Frameset
I frame selezionati per l'esercitazione sono dati dalla chiamata a `getFrameSetForStudent('Fina')`. Il risultato della chamata viene riportato di seguito:

> student: Fina
> * ID:  660      frame: Existence
> * ID: 1710      frame: Mental_activity
> * ID:  111      frame: Education_teaching
> * ID:  250      frame: Immobilization
> * ID: 2231      frame: Response_scenario

In [ ]:
frame_names = ['Existence', 'Mental_activity', 
               'Education_teaching', 'Immobilization', 
               'Response_scenario']

## Annotazione

La fase di annotazione prevede di creare un mapping manuale tra elementi del frame e WordNet sysnet. La funzione `generate_annotations()` genera un file tsv `output/to_annotate.tsv` di aiuto nel processo di annotazione. 

Dopo aver annotato con i rispettivi sysnet id, il file deve essere **rinominato** in `annotations.tsv` e copiato nella directory `data`. Quest'ultimo passaggio è necessario per far funzionare correttamente il resto del codice presente in questo notebook!

In [ ]:
def frame_to_dataframe(frame):
    frame_name = [f"{frame.name}"]
    FEs = [f"{v.name}" for k,v in frame['FE'].items()]
    LUs = [f"{v.name}" for k,v in frame['lexUnit'].items()]

    frame_def = [frame.definition.split('.')[0]] #just the first sentence of the frame definition text
    FEs_def = [v.definition for k,v in frame['FE'].items()]
    LUs_def = [v.definition for k,v in frame['lexUnit'].items()]
    
    index_name = [frame.name] * (1 + 
    len(FEs) + len(LUs)) # index first level
    index_frame = ['NAME'] # second level index for the frame itself
    index_FE = ['FE'] * len(FEs) # second level index for frame elements slots
    index_lu = ['LU'] * len(LUs) # second level index for lexical unit slots


    return pd.DataFrame({'value': frame_name + FEs + LUs, 
                         'definition': frame_def + FEs_def + LUs_def}, 
                         index=[index_name, index_frame + index_FE + index_lu ], 
                         columns=['value', 'definition'])


def generate_annotations(frame_names):
    frames = pd.DataFrame()

    frames = [frame_to_dataframe(fn.frame(name)) for name in frame_names]

    frames_df = pd.concat(frames, axis=0)
    frames_df['synset_ID'] = None
    frames_df.index.names = ['frame', 'slot']
    
    return frames_df

In [ ]:
generate_annotations(frame_names).to_csv('output/to_annotate.tsv', sep='\t')


Dopo aver generato il file di annotazione, bisogna effettuare il mapping **manualmente**.
Il risultato rappresenta il gold standard con cui comparare l'output della procedura di mapping

In [ ]:
annotations  = pd.read_csv('data/annotations.tsv', sep='\t', index_col=[0,1], skipinitialspace=True)

annotations.head(15) 

value  \
frame     slot                     
Existence NAME         Existence   
          FE              Entity   
          FE                Time   
          FE            Duration   
          FE    Inherent_purpose   
          FE               State   
          FE         Explanation   
          FE               Place   
          FE       Circumstances   
          FE           Viewpoint   
          FE          Concessive   
          FE           Depictive   
          LU             exist.v   
          LU         existence.n   
          LU              real.a   

                                                       definition  \
frame     slot                                                      
Existence NAME  An Entity is declared to exist, generally irre...   
          FE    Some entity, abstract or concrete, which is st...   
          FE          A time at which the entity is in existence.   
          FE           The period during which the Entity exists.   
          FE    The reason why the Entity exists.  Generally, ...   
          FE    The condition that the Entity exists in.  'Nat...   
          FE    A situation, force, or entity which brings abo...   
          FE                             Where the Entity exists.   
          FE    Circumstances marks expressions that indicate ...   
          FE    The perspective of an individual who judges wh...   
          FE    This FE signifies that the state of affairs ex...   
          FE    This FE describes a participant of the state o...   
          LU     FN: be real, directly or indirectly perceptible.   
          LU                 COD:  the fact or state of existing.   
          LU    COD: actually existing or occurring in fact; n...   

                         synset_ID  
frame     slot                      
Existence NAME          being.n.01  
          FE           entity.n.01  
          FE             time.n.05  
          FE         duration.n.01  
          FE          purpose.n.01  
          FE            state.n.02  
          FE      explanation.n.01  
          FE         position.n.01  
          FE     circumstance.n.01  
          FE    point_of_view.n.01  
          FE       concessive.a.01  
          FE      delineative.s.01  
          LU            exist.v.01  
          LU            being.n.01  
          LU             real.a.01

## Procedura di Mapping 

La procedura automatica è basata sull'idea di **context overlapping** tra il contesto del frame (un suo slot) e quello del synset preso in considerazione.
FOrmalmente:

$$ \operatorname{overlap(w,s)} = |\operatorname{ctx}(w) \cap \operatorname{ctx}(s)| + 1 $$

Rimangono da definire due punti prima di implementare la procedura:

* come rappresentare il contesto?
* come gestire la polisemia dei termini?

Nel caso più semplice, il contesto può essere rappresentato dal modello *bag-of-words*. Nello specifico:

### Framenet Context

Per un elemento del frame, il contesto è dato dalla bag-of-words della definizione dell'elemento stesso. Vediamo un esempio:

In [ ]:
fncb = mapp.FrameNetContext()

frame = fn.frame(frame_names[0])
print(f"'{frame.name}' frame definition: {frame.definition}")
print()
frame_ctx = fncb.get_context(frame, frame.name, mapp.FrameNetSlotType.NAME)
print(f"'{frame.name}' frame context: {frame_ctx}")

'Existence' frame definition: An Entity is declared to exist, generally irrespective of its position or even the possibility of its position being specified. Time, Duration, Inherent_purpose, and State may also be mentioned.  This frame is to be contrasted with Presence, which describes the existence of an Entity in a particular (and salient) spacio-temporal context, and which also entails the presence of an observer who can detect the existence of the Entity in that context.  'Finally, Poland ceased to exist as a state for hundreds of years.' 'Such laws exist to prevent exactly this kind of fraud.' 'There IS a Santa Claus!'

'Existence' frame context: {'observer', 'frame', 'prevent', 'inherent_purpose', 'presence', "'finally", 'detect', 'ceased', 'hundred', 'contrasted', 'exactly', 'exist', 'salient', 'generally', 'fraud', 'existence', 'irrespective', 'law', 'time', 'declared', 'entity', 'duration', "'there", 'claus', 'context', 'poland', 'santa', 'entail', 'possibility', 'position', 

La classe 'FrameNetContext' permette di estrarre il contesto sia del frame, che da un so FE (frame element) o LU (lexical unit).

### Wordnet Context

Per un WordNet synset, il contesto è dato dalla bag-of-words da:
* definizione della glossa del synset
* frasi di esempio
* le due precedenti ma dei synset in relazione di iperonimia e iponimia.

Vediamo un esempio:

In [ ]:
wncb = mapp.WordNetContext()

synset = wn.synset('being.n.01')
print(f"'{synset.name()}': {synset.definition()}\nexample sentences: {synset.examples()}")
print()
synset_ctx = wncb.get_context(synset)
print(f"'{synset.name()}' frame context: {synset_ctx}")

'being.n.01': the state or fact of existing
example sentences: ['a point of view gradually coming into being', 'laws in existence for centuries']

'being.n.01' frame context: {'point', 'capability', 'condition', 'characterize', 'action', 'fact', 'current', 'existing', 'century', 'main', 'occur', 'believed', 'previous', 'limit', 'alive', 'afterlife', 'religion', 'substance', 'material', 'existence', 'law', 'objectively', 'attribute', 'experience', 'gradually', 'mode', 'course', 'true', 'living', 'coming', 'event', 'eternal', 'individual', 'view', 'respect', 'reality', 'happening', 'characteristic', 'peacefully'}


Rimane un ultimo punto lasciato in sospeso: il fenomeno della polisemia. Dato il valore di un elemento del frame, essenzialmente un termine/lemma, a questo possono corrispondere molteplici sensi:

In [ ]:
time_FE = annotations.loc[('Existence','FE'),'value'].iloc[1]
wn.synsets(time_FE)

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


[Synset('time.n.01'),
 Synset('time.n.02'),
 Synset('time.n.03'),
 Synset('time.n.04'),
 Synset('time.n.05'),
 Synset('time.n.06'),
 Synset('clock_time.n.01'),
 Synset('fourth_dimension.n.01'),
 Synset('meter.n.04'),
 Synset('prison_term.n.01'),
 Synset('clock.v.01'),
 Synset('time.v.02'),
 Synset('time.v.03'),
 Synset('time.v.04'),
 Synset('time.v.05')]

La procedura di mapping seleziona il senso che **massimizza** la misura di overlapping. In formula:


$$ s^* = \operatorname*{max}_{\substack{s \in \operatorname*{synsets(w)}}}\operatorname*{overlap(w,s)} $$

Di seguito vediamo un esecuzione della procedura su alcune annotazioni

In [ ]:
def map_annotation(df_row, mapper):
    """
    This is just an helper function to deal with DataFrame.apply method.map
    The function just help to call mapping.FrameToSynsetMapper.map method with the right
    parameters since apply() give in input a DataFrame.row istance
    """
    frame_name = df_row.name[0]
    slot_value = df_row['value']
    slot_type = mapp.FrameNetSlotType[df_row.name[1]]
    return mapper.map(frame_name, slot_value, slot_type)

In [ ]:
predictions = annotations[['value','synset_ID']].copy()  # discard definitions

fncb = mapp.FrameNetContext()
wncb = mapp.WordNetContext()
mapper = mapp.FrameToSynsetMapper(fncb, wncb)

predictions['system_synset'] = predictions.apply(map_annotation, mapper=mapper, axis=1)
#predictions['system_synset'].fillna('',inplace=True) 
predictions.head(15)

value           synset_ID           system_synset
frame     slot                                                              
Existence NAME         Existence          being.n.01              being.n.01
          FE              Entity         entity.n.01             entity.n.01
          FE                Time           time.n.05               time.n.05
          FE            Duration       duration.n.01           duration.n.01
          FE    Inherent_purpose        purpose.n.01                    None
          FE               State          state.n.02              state.n.02
          FE         Explanation    explanation.n.01        explanation.n.01
          FE               Place       position.n.01  topographic_point.n.01
          FE       Circumstances   circumstance.n.01       circumstance.n.01
          FE           Viewpoint  point_of_view.n.01      point_of_view.n.01
          FE          Concessive     concessive.a.01         concessive.a.01
          FE           Depictive    delineative.s.01        delineative.s.01
          LU             exist.v          exist.v.01              exist.v.01
          LU         existence.n          being.n.01              being.n.01
          LU              real.a           real.a.01               real.a.01

## Risultati

In [ ]:
def accuracy(true, predicted):
    correct_predictions = sum([true_sense.lower() == predicted_sense.lower() for true_sense, predicted_sense in zip(true, predicted)
    if true_sense and predicted_sense])
    return correct_predictions / len(true)

In [ ]:
acc = accuracy(predictions['synset_ID'], predictions['system_synset'])
print(f"Total accuracy achieved: {acc}")

Total accuracy achieved: 0.5196078431372549


In [ ]:
for frame_name, frame_df in predictions.groupby('frame'):
    acc = accuracy(frame_df['synset_ID'], frame_df['system_synset'])
    print(f"Frame: {frame_name} with accuracy: {acc}")

Frame: Education_teaching with accuracy: 0.5510204081632653
Frame: Existence with accuracy: 0.7647058823529411
Frame: Immobilization with accuracy: 0.3125
Frame: Mental_activity with accuracy: 0.45454545454545453
Frame: Response_scenario with accuracy: 0.3333333333333333


Come è possibile osservare dall'output delle precedenti celle, l'accuracy totale raggiunta dal sistema è del $\approx 0.55\%$ con un deviazione std. dello $\approx 0.16\%$ calcolata sull'accuracy individuale dei 5 frame del frameset.

Risulta interessante analizzare i risultati ottenuti su i frame che hanno ottenuto la massima e la minima accuratezza, rispettivamente: *"Existence"* e *"Response_scenario"*. 

Da una prima analisi quantitativa e qualitativa della composizione dei due frame, possiamo suppore che la differenza di performance sia dovuta principalmente a:

* Ricchezza delle annotazioni lessicali. Il numero di FE e LU del frame "Existence" è maggiore rispetto al frame "Response_scenario".
* Ridotto "**impedance mismatch**" tra il frame "Existence" e i sensi presenti in WordNet. Il frame "Response_scenario" descrive una situazione stereotipizzata al contrario del frame "Existence" il quale presenta concetti molto generali che ben si adattano alla natura di WordNet, e quindi ne facilitano il mapping.

Quest'ultimo punto, è stato empiricamente osservato anche in fase di annotazione manuale, in moltepliici casi.

In [ ]:
fes_existence = len(fn.frame('Existence')['FE'])
fes_response = len(fn.frame('Response_scenario')['FE'])
print(f"# FEs {fes_existence} vs {fes_response}")

lus_existence = len(fn.frame('Existence')['lexUnit'])
lus_response = len(fn.frame('Response_scenario')['lexUnit'])
print(f"# LUs {lus_existence} vs {lus_response}")

# FEs 11 vs 8
# LUs 5 vs 0


Bisogna comunque porre molta attenzione a generalizzare la precedente osservazione in quanto il numero di campioni analizzati (frame) non è statisticamente significante. Le conclusioni riportate però possono porre le basi per una successiva (e più accurata) investigazione dell'errore.  

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=afb22156-bb61-4d65-847d-18db79c0d4d2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>